In [8]:
from mmCif import mmcifIO
#from mmcifIO import mmcifIO
# load the mmCIF parser
cfr = mmcifIO.CifFileReader(input='dictionary')
# read an mmCIF file
cif_file = cfr.read("1a3n.cif", output='cif_file')


CifFile: <CifFile "1a3n.cif">


mmCIF files are organised into 
DataBlocks --> Categories --> Items
The data we want in PDB mmCIF files is in the first (and in most cases) the only DataBlock

In [9]:
mmCIF_data = cif_file.getDataBlocks()[0]

Entity information is stored witin the mmCIF category entity

In [13]:
entity_category = mmCIF_data.getCategory('entity')

print("Entity information:", entity_category)

Entity information: <Category "_entity" with items ['id', 'type', 'src_method', 'pdbx_description', 'formula_weight', 'pdbx_number_of_molecules', 'pdbx_ec', 'pdbx_mutation', 'pdbx_fragment', 'details']>


In [ ]:
The items within a category can be 

In [18]:
entity_items = entity_category.getItemNames()
print(entity_items)

['id', 'type', 'src_method', 'pdbx_description', 'formula_weight', 'pdbx_number_of_molecules', 'pdbx_ec', 'pdbx_mutation', 'pdbx_fragment', 'details']


Entity names are stored in _entity.pdbx_description item

In [16]:
entity_names = entity_category.getItem('pdbx_description').value
print(entity_names)

['HEMOGLOBIN (ALPHA CHAIN)', 'HEMOGLOBIN (BETA CHAIN)', 'PROTOPORPHYRIN IX CONTAINING FE', 'water']


only some of these are polymers

In [17]:
entity_types = entity_category.getItem('type').value
print(entity_types)

['polymer', 'polymer', 'non-polymer', 'water']


Their IDs are available in the ID item

In [37]:
entity_ids = entity_category.getItem('id').value
print(entity_ids)

['1', '2', '3', '4']


We can get data for all items with a simple function

In [38]:
def get_cat_dict_values(category):
    result = dict()
    category_object = mmCIF_data.getCategory(category)
    items = category_object.getItemNames()
    if items:
        for item in items:
            values = category_object.getItem(item).value
            # the mmCIF parser will return a string for a single value - here we convert it to a list so its consistent
            if not isinstance(values, list):
               values = [values]
            result[item] = values
    return result

In [40]:
category_data = get_cat_dict_values('entity')
print(category_data)

{'id': ['1', '2', '3', '4'], 'type': ['polymer', 'polymer', 'non-polymer', 'water'], 'src_method': ['nat', 'nat', 'syn', 'nat'], 'pdbx_description': ['HEMOGLOBIN (ALPHA CHAIN)', 'HEMOGLOBIN (BETA CHAIN)', 'PROTOPORPHYRIN IX CONTAINING FE', 'water'], 'formula_weight': ['15150.353', '15890.198', '616.487', '18.015'], 'pdbx_number_of_molecules': ['2', '2', '4', '451'], 'pdbx_ec': ['?', '?', '?', '?'], 'pdbx_mutation': ['?', '?', '?', '?'], 'pdbx_fragment': ['?', '?', '?', '?'], 'details': ['?', '?', '?', '?']}


We can change this to a list of dictionaries if this is more convient

In [43]:
def get_cat_list_of_dict(category):
    result = list()
    category_object = mmCIF_data.getCategory(category)
    items = category_object.getItemNames()
    values_in_lists = get_cat_dict_values(category)
    if items and values_in_lists:
        num_values = len(values_in_lists[items[0]])
        range_of_values = range(num_values)
        for row in range_of_values:
            row_dict = dict()
            for item in values_in_lists:
                item_value = values_in_lists[item][row]
                row_dict[item] = item_value
            result.append(row_dict)
    return result

In [44]:
category_list_of_dicts = get_cat_list_of_dict('entity')
print(category_list_of_dicts)

[{'id': '1', 'type': 'polymer', 'src_method': 'nat', 'pdbx_description': 'HEMOGLOBIN (ALPHA CHAIN)', 'formula_weight': '15150.353', 'pdbx_number_of_molecules': '2', 'pdbx_ec': '?', 'pdbx_mutation': '?', 'pdbx_fragment': '?', 'details': '?'}, {'id': '2', 'type': 'polymer', 'src_method': 'nat', 'pdbx_description': 'HEMOGLOBIN (BETA CHAIN)', 'formula_weight': '15890.198', 'pdbx_number_of_molecules': '2', 'pdbx_ec': '?', 'pdbx_mutation': '?', 'pdbx_fragment': '?', 'details': '?'}, {'id': '3', 'type': 'non-polymer', 'src_method': 'syn', 'pdbx_description': 'PROTOPORPHYRIN IX CONTAINING FE', 'formula_weight': '616.487', 'pdbx_number_of_molecules': '4', 'pdbx_ec': '?', 'pdbx_mutation': '?', 'pdbx_fragment': '?', 'details': '?'}, {'id': '4', 'type': 'water', 'src_method': 'nat', 'pdbx_description': 'water', 'formula_weight': '18.015', 'pdbx_number_of_molecules': '451', 'pdbx_ec': '?', 'pdbx_mutation': '?', 'pdbx_fragment': '?', 'details': '?'}]


sequences are stored in entity_poly category
This category has an item entity_id which is a foreign key to entity.id

In [47]:
entity_poly_data = get_cat_list_of_dict('entity_poly')
print(entity_poly_data)

[{'entity_id': '1', 'type': 'polypeptide(L)', 'nstd_linkage': 'no', 'nstd_monomer': 'no', 'pdbx_seq_one_letter_code': 'VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNAL\nSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR', 'pdbx_seq_one_letter_code_can': 'VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNAL\nSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR', 'pdbx_strand_id': 'A,C', 'pdbx_target_identifier': '?'}, {'entity_id': '2', 'type': 'polypeptide(L)', 'nstd_linkage': 'no', 'nstd_monomer': 'no', 'pdbx_seq_one_letter_code': 'VHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDN\nLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH', 'pdbx_seq_one_letter_code_can': 'VHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDN\nLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH', 'pdbx_strand_id': 'B,D', 'pdbx_target_identifi

The canonical sequence is stored in _entity_poly.pdbx_seq_one_letter_code_can

coordinates are stored in atom_site. 
This category has an item entity_id which is also a foreign key to the entity category

In [48]:
atom_site_data = get_cat_list_of_dict('atom_site')
print(atom_site_data)

[{'group_PDB': 'ATOM', 'id': '1', 'type_symbol': 'N', 'label_atom_id': 'N', 'label_alt_id': '.', 'label_comp_id': 'VAL', 'label_asym_id': 'A', 'label_entity_id': '1', 'label_seq_id': '1', 'pdbx_PDB_ins_code': '?', 'Cartn_x': '10.720', 'Cartn_y': '19.523', 'Cartn_z': '6.163', 'occupancy': '1.00', 'B_iso_or_equiv': '21.36', 'pdbx_formal_charge': '?', 'auth_seq_id': '1', 'auth_comp_id': 'VAL', 'auth_asym_id': 'A', 'auth_atom_id': 'N', 'pdbx_PDB_model_num': '1'}, {'group_PDB': 'ATOM', 'id': '2', 'type_symbol': 'C', 'label_atom_id': 'CA', 'label_alt_id': '.', 'label_comp_id': 'VAL', 'label_asym_id': 'A', 'label_entity_id': '1', 'label_seq_id': '1', 'pdbx_PDB_ins_code': '?', 'Cartn_x': '10.228', 'Cartn_y': '20.761', 'Cartn_z': '6.807', 'occupancy': '1.00', 'B_iso_or_equiv': '24.26', 'pdbx_formal_charge': '?', 'auth_seq_id': '1', 'auth_comp_id': 'VAL', 'auth_asym_id': 'A', 'auth_atom_id': 'CA', 'pdbx_PDB_model_num': '1'}, {'group_PDB': 'ATOM', 'id': '3', 'type_symbol': 'C', 'label_atom_id': '

we could get the coordinates for only entity 1

In [51]:
result_data = []
for row in atom_site_data:
    entity_id = row.get('label_entity_id')
    if entity_id == '1':
        result_data.append(row)
        
print(result_data)

[{'group_PDB': 'ATOM', 'id': '1', 'type_symbol': 'N', 'label_atom_id': 'N', 'label_alt_id': '.', 'label_comp_id': 'VAL', 'label_asym_id': 'A', 'label_entity_id': '1', 'label_seq_id': '1', 'pdbx_PDB_ins_code': '?', 'Cartn_x': '10.720', 'Cartn_y': '19.523', 'Cartn_z': '6.163', 'occupancy': '1.00', 'B_iso_or_equiv': '21.36', 'pdbx_formal_charge': '?', 'auth_seq_id': '1', 'auth_comp_id': 'VAL', 'auth_asym_id': 'A', 'auth_atom_id': 'N', 'pdbx_PDB_model_num': '1'}, {'group_PDB': 'ATOM', 'id': '2', 'type_symbol': 'C', 'label_atom_id': 'CA', 'label_alt_id': '.', 'label_comp_id': 'VAL', 'label_asym_id': 'A', 'label_entity_id': '1', 'label_seq_id': '1', 'pdbx_PDB_ins_code': '?', 'Cartn_x': '10.228', 'Cartn_y': '20.761', 'Cartn_z': '6.807', 'occupancy': '1.00', 'B_iso_or_equiv': '24.26', 'pdbx_formal_charge': '?', 'auth_seq_id': '1', 'auth_comp_id': 'VAL', 'auth_asym_id': 'A', 'auth_atom_id': 'CA', 'pdbx_PDB_model_num': '1'}, {'group_PDB': 'ATOM', 'id': '3', 'type_symbol': 'C', 'label_atom_id': '